In [ ]:
%%sh
pip -q install transformers datasets ipywidgets mlflow --upgrade
curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
sudo yum install git-lfs -y
git lfs install

In [ ]:
%matplotlib inline

In [ ]:
import os

import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login
from matplotlib import pyplot as plt
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

import mlflow

In [ ]:
notebook_login()

In [ ]:
!git config --global credential.helper store

In [ ]:
train_dataset, test_dataset = load_dataset("imdb", split=["train", "test"])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

In [ ]:
metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# https://huggingface.co/docs/transformers/v4.20.1/en/main_classes/callback#transformers.integrations.MLflowCallback

os.environ["MLFLOW_EXPERIMENT_NAME"] = "trainer-mlflow-demo"
os.environ["MLFLOW_FLATTEN_PARAMS"] = "1"
# os.environ["MLFLOW_TRACKING_URI"]=""
# os.environ["HF_MLFLOW_LOG_ARTIFACTS"]="1"

In [ ]:
training_args = TrainingArguments(
    hub_model_id="juliensimon/distilbert-imdb-mlflow",
    num_train_epochs=1,
    output_dir="./output",
    logging_steps=500,
    save_strategy="epoch",
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
mlflow.end_run()

In [ ]:
trainer.push_to_hub()

In [ ]:
%%sh
cp -r mlruns output
cd output
git add mlruns
git commit -m 'Add MLflow run'
git push

In [ ]:
!ls -l mlruns/1

In [ ]:
run = "1/1a4ab6fd80784bd6a83b447fd01a45ca"

In [ ]:
def plot_metric(metric_name):
    metric = pd.read_csv(
        f"mlruns/{run}/metrics/{metric_name}", sep=" ", names=["timestamp", "value", "steps"]
    )
    plt.plot(metric["steps"], metric["value"])

In [ ]:
plot_metric("loss")

In [ ]:
plt.clf()
plot_metric("learning_rate")